In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import torch
import cv2
import imageio
import numpy as np
from PIL import Image  
from denku import show_images

from kandimate.utils.util import save_videos_grid

In [2]:
device = torch.device('cuda')
precision = torch.float16
model_path = '../models/interpolation-models/film_net_fp16.pt'

In [3]:
model = torch.jit.load(model_path, map_location='cpu')
model.eval().to(device=device, dtype=precision);

In [4]:
gif_path = './samples/example.gif'

gif_file = imageio.get_reader(gif_path)
images = [np.array(x) for x in gif_file]

In [5]:
def pil_list_to_torch(images):
    tensor = [torch.from_numpy(x) for x in images]
    tensor = [x.permute(2, 0, 1).unsqueeze(0) for x in tensor]
    tensor = [x.float() / 255.0 for x in tensor]
    tensor = torch.cat(tensor)
    return tensor

def torch_to_np(tensor):
    images = tensor.permute(0, 2, 3, 1).numpy()
    images = np.clip(images * 255, 0, 255)
    images = images.astype(np.uint8)
    return images

tensor = pil_list_to_torch(images)

In [6]:
interpolation_frames = 3

In [7]:
prev_tensor = tensor[0:-1:1].to(precision).to(device)
next_tensor = tensor[1::1].to(precision).to(device)
dt = prev_tensor.new_full((1, 1), 0.5)

prev_tensor.shape, next_tensor.shape

(torch.Size([15, 3, 768, 768]), torch.Size([15, 3, 768, 768]))

In [8]:
with torch.no_grad():
    dt_images_mid = model(prev_tensor, next_tensor, dt).cpu()
    generated_prev = torch_to_np(dt_images_mid)
    
if interpolation_frames > 1:
    with torch.no_grad():
        dt_images_prev = model(prev_tensor.to(device), dt_images_mid.to(device), dt).cpu()
        dt_images_next = model(dt_images_mid.to(device), next_tensor.to(device), dt).cpu()
        
    generated_mid = torch_to_np(dt_images_mid)
    generated_next = torch_to_np(dt_images_next)

/home/user/Projects/denk_baseline/venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1501: UserWarning: operator() sees varying value in profiling, ignoring and this should be handled by GUARD logic (Triggered internally at ../third_party/nvfuser/csrc/parser.cpp:3777.)
  return forward_call(*args, **kwargs)
/home/user/Projects/denk_baseline/venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1501: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1003.)
  return forward_call(*args, **kwargs)
/home/user/Projects/denk_baseline/venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1501: UserWarning: operator() profile_node %511 : int[] = prim::profile_ivalue[profile_failed="varying profile values"](%509)
 does not have profile information (Triggered internally at ../third_party/nvfuser/csrc/graph_fuser.cpp:104.)
  retur

In [9]:
extended_images = [images[0]]

if interpolation_frames == 1:
    for mid_img, orig_img in zip(generated_mid, images[1:]):
        extended_images.extend([mid_img, orig_img])
elif interpolation_frames == 2:
    for prev_img, next_img, orig_img in zip(generated_prev, generated_next, images[1:]):
        extended_images.extend([prev_img, next_img, orig_img])
elif interpolation_frames == 3:
    for prev_img, mid_img, next_img, orig_img in zip(generated_prev, generated_mid, generated_next, images[1:]):
        extended_images.extend([prev_img, mid_img, next_img, orig_img])

In [10]:
imageio.mimsave('./samples/test.gif', extended_images, duration=60)

In [15]:
def resize_gif(in_path, out_path, img_h=512, img_w=512, duration=60):
    gif_file = imageio.get_reader(in_path)
    images = [cv2.resize(x, (img_w, img_h)) for x in gif_file]
    imageio.mimsave(out_path, images, duration=duration)
    
resize_gif(
    in_path='./samples/test.gif', 
    out_path='./samples/resized.gif',
    img_h=384, 
    img_w=384,
    duration=90,
)